In [1]:
import sys
import os

# Add the parent directory of 'vegetation-period-NDVI' to the sys.path
sys.path.append(os.path.abspath(os.path.join('..')))


In [2]:
import ee
import geemap
from time_series import extract_time_ranges, get_harmonic_ts
from vegetation_period_extraction import (
    get_crop_veg_period,
    create_binary_ndvi_indicator,
    create_binary_mask,
)
from typing import List

In [3]:
ee.Initialize(project="thurgau-irrigation")

## Exporting the vegetation period images as an asset to the project

### Define the Area of Interest

In [4]:
cantonal_borders_asset = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

aoi_name = "Thurgau"

### Define the years of interest

In [8]:
years = [2019, 2020, 2021, 2022, 2023]

### Export the Vegetation Periods images to the project assets

In [9]:
for year in years:

    print(f"Processing year {year}")

    time_intervals = extract_time_ranges([f"{year}-03-01", f"{year}-10-31"], 15)

    image_to_export = get_crop_veg_period(year, aoi_buffered, time_intervals)

    task_name = f"Thurgau_crop_veg_period_{year}"
    asset_id = f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{year}"

    bands_to_export = ["firstStart", "firstEnd", "secondStart", "secondEnd", "isDoubleCropping"]

    image_to_export = image_to_export.select(bands_to_export)


    task = ee.batch.Export.image.toAsset(
        image=image_to_export,
        description=task_name,
        assetId=asset_id,
        region=aoi_buffered,
        scale=10,
        maxPixels=1e13,
    )

    task.start()

    print(f"Exporting {task_name} to {asset_id}")

Processing year 2019
Exporting Thurgau_crop_veg_period_2019 to projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2019
Processing year 2020
Exporting Thurgau_crop_veg_period_2020 to projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2020
Processing year 2021
Exporting Thurgau_crop_veg_period_2021 to projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2021
Processing year 2022
Exporting Thurgau_crop_veg_period_2022 to projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2022
Processing year 2023
Exporting Thurgau_crop_veg_period_2023 to projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_2023


## General Code Testing. Ignore this part

In [6]:
year = 2019

# Define the time range for analysis
start_date = f"{year}-03-01"
end_date = f"{year}-10-31"

# Create time intervals
time_intervals = extract_time_ranges([start_date, end_date], 15)

# Get the vegetation period estimation
veg_period_image = get_crop_veg_period(year, aoi_buffered, time_intervals)

In [ ]:
# type(veg_period_image)

In [7]:
double_band = veg_period_image.select('isDoubleCropping')

# Create a map
Map = geemap.Map(zoom=10)
Map.centerObject(aoi_buffered)
# Add the 'double' band to the map
Map.addLayer(double_band, {'min': 0, 'max': 2, 'palette': ['blue', 'green', 'red']}, 'Double Band')

# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
filtered_sentinel_data = get_harmonic_ts(year=year, aoi=aoi_buffered, time_intervals=time_intervals)

In [ ]:
filtered_sentinel_data

In [ ]:
Map = geemap.Map()


# Add the layer to the map.
ndvi_image = ee.Image(filtered_sentinel_data.get("fitted_data").toList(99).get(7))
Map.centerObject(aoi_buffered, 13)
ndvi_params = {'bands': ['fitted'], 'min': 0, 'max': 1, 'palette': ['white', 'green']}
Map.addLayer(ndvi_image, ndvi_params, 'NDVI FIRST', True)

# Display the map.
Map

# Testing Downscaling script

In [5]:
from data_loading import load_sentinel2_data
from utils.composites import harmonized_ts

In [6]:
s2collection = load_sentinel2_data(year=2019, aoi=aoi_buffered)

In [7]:
# Get date of first image in s2collection
first_image = s2collection.first()
date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
print(f"First image date: {date}")

First image date: 2019-01-15


In [8]:
year = 2019
time_intervals = extract_time_ranges([f"{year}-01-01", f"{year}-12-31"], 30.5)

bands = ["B4", "B8", "B11", "B12"]  # The bands you want to process

harmonic_result = harmonized_ts(
    masked_collection=s2collection,
    band_list=bands,
    time_intervals=time_intervals,
    options={"band_name": "B4", "agg_type": "median"}
)

# Print the band names of the result for verification
result_bands = harmonic_result.first().bandNames().getInfo()
print(f"Resulting band names: {result_bands}")

Resulting band names: ['B4', 'B8', 'B11', 'B12']


In [13]:
# Get number of images in harmonic_result
num_images = harmonic_result.size().getInfo()
print(f"Number of images in harmonic_result: {num_images}")

for i in range(num_images):
    image = ee.Image(harmonic_result.toList(num_images).get(i))
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    print(f"Image {i} date: {date}")

Number of images in harmonic_result: 12
Image 0 date: 2019-01-17
Image 1 date: 2019-02-15
Image 2 date: 2019-03-17
Image 3 date: 2019-04-16
Image 4 date: 2019-05-17
Image 5 date: 2019-06-16
Image 6 date: 2019-07-17
Image 7 date: 2019-08-17
Image 8 date: 2019-09-16
Image 9 date: 2019-10-17
Image 10 date: 2019-11-16
Image 11 date: 2019-12-17


In [10]:
Map = geemap.Map()


# Add the layer to the map.
ndvi_image = ee.Image(harmonic_result.first())
Map.centerObject(aoi_buffered, 13)
ndvi_params = {'bands': ['B4']}
Map.addLayer(ndvi_image, ndvi_params, 'B4 band', True)

# Display the map.
Map

Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…